# Creating a Real-Time Inferencing Service
Now it's time to deploy a model as a real-time service that clients can use to get predictions from new data.

Lenovo Engineer kaustav biswas: 9831059668

### 1. Connect to Your Workspace

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 2. Train and Register a Model

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('../mslearn-aml-labs/data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

### 3. Deploy a Model as a Web Service
This model could be used in a production environment such as a doctor's surgery where only patients deemed to be at risk need to be subjected to a clinical test for diabetes. To support this scenario, you will deploy the model as a web service.

#### 3.1 Pick a model from a list of models

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

#### 3.2 Create a folder for those.

In [ ]:
import os
folder_name = 'diabetes_service'

#create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created')

#### 3.3 Create entry script
The web service where we deploy the model will need some Python code to   
    1.  load the input data,   
    2.  get the model from the workspace,   
    3.  and generate and return predictions.

In [ ]:
%%writefile $folder_name/score_diabetes.py
import joblib
import json
import numpy as np
from azureml.core.model import Model

#called when the service is loaded
def init():
    global model
    #Get the path to the depoyed model and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


#called when a request is received
def run(raw_data):
    #Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    #Get a prediction from the model
    predictions = model.predict(data)
    #Get the corresponding classname for each of the prediction
    classnames = ['non-diabetic', 'diabetic']
    predicted_classes = []

    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    
    return json.dumps(predicted_classes)








### 3.4 Create yml file to resolve container dependency

The web service will be **hosted in a container**, and the container will need to install any required Python dependencies when it gets initialized. 
In this case, our scoring code requires scikit-learn,

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

#Add the dependency
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file, "w") as f:
    f.write(myenv.serialize_to_string())
print('saved dependency info in', env_file)


#print the .yml file
with open(env_file, "r") as f:
    print(f.read())





### 3.5 Deployment steps
We'll deploy the container a service named diabetes-service using  **Azure Container Instance (ACI)** . 
The deployment process includes the following steps:  

1. Define an inference configuration  

2. Define a deployment configuration   

3. Deploy the model as a web service.  

4. Verify the status of the deployed service.  


In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

for webservice_name in ws.webservices:
    print(webservice_name)

### Deploy model with Azure Kubernetes Service (AKS) cluster
 If we are deploying to an AKS cluster, we must create the cluster and a compute target for it before deploying:

 The code to configure an ACI deployment is similar, except that we do not need to 
 explicitly create an ACI compute target, and we must use the deploy_configuration class from the **azureml.core.webservice.AciWebservice** namespace.  
 Similarly, we can use the **azureml.core.webservice.LocalWebservice** namespace to 
 configure a local Docker-based service.

In [ ]:
#from azureml.core.compute import ComputeTarget, AksCompute

#cluster_name = 'aks-cluster'
#compute_config = AksCompute.provisioning_configuration(location='eastus')
#production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
#production_cluster.wait_for_completion(show_output=True)


#from azureml.core.webservice import AksWebservice

#classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

### 4. Consuming a real-time inferencing service
#### 4.1 For **testing**, we can use the **Azure Machine Learning SDK** 
to call a web service through the run method of a WebService object that references the deployed service. 
Typically, we send data to the run method in JSON format with the following structure:


In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

#### 4.2 In production
Most client applications will not include the Azure Machine Learning SDK, 
and will **consume the service** through its **REST** interface. 
We can determine the endpoint of a deployed service in Azure machine Learning studio, 
or by retrieving the **scoring_uri** property of the **Webservice** object in the SDK, like this:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Now that we know the endpoint URI, an application can simply make an HTTP request, 
sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )